In [ ]:
# import library

import tensorflow as tf
import pandas as pd
from PIL import Image
import os
import numpy as np
import matplotlib.pyplot as plt
import os
import pathlib
import random

In [2]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#connect to dataset folder
path = '/content/drive/MyDrive/Mias Mammography Database/all-mias'
data_dir = pathlib.Path(path)

In [ ]:
class_names = np.array(sorted([item.name for item in data_dir.glob("*")]))
class_names


In [ ]:
!pip install colorama


In [ ]:

import numpy as np
import pandas as pd
import cv2
from PIL import Image
import scipy

import tensorflow as tf
from tensorflow.keras.applications import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.losses import *
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.callbacks import *
from tensorflow.keras.preprocessing.image import *
from tensorflow.keras.utils import *
# import pydot

from sklearn.metrics import *
from sklearn.model_selection import *
import tensorflow.keras.backend as K

from tqdm import tqdm, tqdm_notebook
from colorama import Fore
import json
import matplotlib.pyplot as plt
import seaborn as sns
from glob import glob
from skimage.io import *
%config Completer.use_jedi = False
import time
from sklearn.decomposition import PCA
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import lightgbm as lgb
import xgboost as xgb

print("All modules have been imported")

In [ ]:
# import pandas as pd
# from google.colab import drive

# # Mount Google Drive
# drive.mount('/content/drive')

# # Define file path
# file_path = '/content/drive/MyDrive/Mias Mammography Database/Info.txt'

# # Read the data
# info = pd.read_csv(file_path, sep='\s+', engine='python', on_bad_lines='skip')

# info.SEVERITY.fillna(0)
# # Display info after modifications
# print(info.head())

import pandas as pd
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive', force_remount=True)

# Define file path
file_path = '/content/drive/MyDrive/Mias Mammography Database/Info.txt'

# Read the data
info = pd.read_csv(file_path, sep='\s+', engine='python', on_bad_lines='skip')

# Check if 'Unnamed: 7' column exists and drop it
if 'Unnamed: 7' in info.columns:
    info = info.drop('Unnamed: 7', axis=1)

# Fill missing values in SEVERITY with 0
info['SEVERITY'] = info['SEVERITY'].fillna(0)

# Display info after modifications
print(info.head())



In [ ]:
sns.set_style('darkgrid')
fig, (ax1, ax2) = plt.subplots(1,2,figsize=(15,5))
sns.barplot(x=info.BG.unique(),y=info.BG.value_counts(),palette='Blues_r',ax=ax1)
sns.barplot(x=info.CLASS.unique(),y=info.CLASS.value_counts(),palette='Blues_r',ax=ax2)


In [ ]:

import glob
import numpy as np
import matplotlib.pyplot as plt
# Load and resize images
x = []
for filename in sorted(glob.glob(os.path.join(data_dir, "*.pgm"))):
    img = cv2.imread(filename, cv2.IMREAD_GRAYSCALE)  # Read as grayscale
    img = cv2.resize(img, (128, 128))
    x.append(img)

# Display images in a grid
fig = plt.figure(figsize=(15, 15))
columns = 3
rows = 3

for i in range(1, columns * rows + 1):
    fig.add_subplot(rows, columns, i)
    plt.imshow(x[i - 1], cmap='gray')
    plt.axis('off')

plt.show()


In [ ]:
import numpy as np
lable_info=read_lable()
image_info=read_image()
ids=lable_info.keys()
del lable_info['Truth-Data:']
X=[]
Y=[]
for id in ids:
    for angle in range(0,no_angles,8):
        X.append(image_info[id][angle])
        Y.append(lable_info[id][angle])
X=np.array(X)
Y=np.array(Y)
Y=to_categorical(Y,2)
x_train, x_test1, y_train, y_test1 = train_test_split(X, Y, test_size=0.3, random_state=42)
x_val, x_test, y_val, y_test = train_test_split(x_test1, y_test1, test_size=0.3, random_state=42)
print(len(x_train),len(x_val),len(x_test))

In [ ]:
# Image Augmentation
no_angles = 360
url = '/content/drive/MyDrive/Mias Mammography Database/all-mias/'

def save_dictionary(path,data):
        print('saving catalog...')
        #open('u.item', encoding="utf-8")
        import json
        with open(path,'w') as outfile:
            json.dump(str(data), fp=outfile)
        # save to file:
        print(' catalog saved')

In [ ]:
# train_test_split_datagen=ImageDataGenerator("augmentations such as flip,brightness range,etc....")
# val_datagen=ImageDataGenerator("augmentations such as flip,brightness range,etc....")
# test_datagen=ImageDataGenerator("augmentations such as flip,brightness range,etc....")
def read_image():
        print("Reading images")
        import cv2
        info = {}
        for i in range(322):
            if i<9:
                image_name='mdb00'+str(i+1)
            elif i<99:
                image_name='mdb0'+str(i+1)
            else:
                image_name = 'mdb' + str(i+1)
            image_address= url+image_name+'.pgm'
            img = cv2.imread(image_address,1)
            img = cv2.resize(img, (128,128))
            rows, cols,channel = img.shape
            info[image_name]={}
            for angle in range(0,no_angles,8):
                M = cv2.getRotationMatrix2D((cols / 2, rows / 2), angle, 1)
                img_rotated = cv2.warpAffine(img, M, (cols, rows))
                info[image_name][angle]=img_rotated
        return (info)


In [ ]:
def read_lable():
    print("Reading labels")
    filename = url+'Info.txt'
    text_all = open(filename).read()
    #print(text_all)
    lines=text_all.split('\n')
    info={}
    for line in lines:
        words=line.split(' ')
        if len(words)>3:
            if (words[3] == 'B'):
                info[words[0]] = {}
                for angle in range(0,no_angles,8):
                    info[words[0]][angle] = 0
            if (words[3] == 'M'):
                info[words[0]] = {}
                for  angle in range(0,no_angles,8):
                    info[words[0]][angle] = 1
    return (info)

Training, Validation & Testing

In [ ]:
#Callbacks
c2=tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    min_delta=0,
    patience=6,
    mode="auto",
    baseline=None,
    restore_best_weights=True,
)

c3=tf.keras.callbacks.ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.1,
    patience=6,
    mode="auto",
    min_delta=0.0001,
    cooldown=0,
    min_lr=0.001
)
nClasses=2

Data Imbalance

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Define the function to read label information
def read_label():
    print("Reading labels")
    filename = url + 'Info.txt'  # Update the file path accordingly
    with open(filename, 'r') as file:
        lines = file.readlines()
    data = []
    for line in lines[1:]:  # Skip the header line
        words = line.strip().split()
        if len(words) > 2:  # Ensure the line has sufficient information
            image_id = words[0]
            cls = words[2]
            # Convert non-NORM classes to 'Cancer' and NORM to 'Normal'
            if cls != 'NORM':
                cls = 'Cancer'
            else:
                cls = 'Normal'
            data.append([image_id, cls])
    columns = ['REFNUM', 'CLASS']
    label_df = pd.DataFrame(data, columns=columns)
    return label_df

# Read label information from the file
label_df = read_label()

# Plot the class distribution
plt.figure(figsize=(6, 4))
sns.countplot(data=label_df, x='CLASS')
plt.title('Class Distribution')
plt.xlabel('Class')
plt.ylabel('Count')
plt.show()

# Check imbalance
class_counts = label_df['CLASS'].value_counts()
print("Class Distribution:")
print(class_counts)

imbalance_threshold = 0.4  # tolerance data 40%
minority_class_ratio = class_counts.min() / class_counts.max()

if minority_class_ratio < imbalance_threshold:
    print("\nThe data is imbalanced.")
else:
    print("\nThe data is balanced.")


In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Define the function to read label information
def read_label():
    print("Reading labels")
    filename = url + 'Info.txt'  # Update the file path accordingly
    with open(filename, 'r') as file:
        lines = file.readlines()
    data = []
    for line in lines[1:]:  # Skip the header line
        words = line.strip().split()
        if len(words) > 6:  # Ensure the line has sufficient information
            image_id = words[0]
            bg = words[1]
            classification = words[2]
            try:
                x = int(words[4])
                y = int(words[5])
                radius = int(words[6])
            except ValueError:
                # Handle non-numeric values or missing data
                continue
            severity = words[3]
            data.append([image_id, bg, classification, x, y, radius, severity])
    columns = ['REFNUM', 'BG', 'CLASS', 'X', 'Y', 'RADIUS', 'SEVERITY']
    label_df = pd.DataFrame(data, columns=columns)
    return label_df

# Read label information from the file
label_df = read_label()

# Check imbalance by 'SEVERITY'
severity_counts = label_df['SEVERITY'].value_counts()
print("Severity Counts:")
print(severity_counts)

# Plot severity distribution
plt.figure(figsize=(8, 6))
sns.countplot(data=label_df, x='SEVERITY', palette='Set2')
plt.title('Severity Distribution')
plt.xlabel('Severity')
plt.ylabel('Count')
plt.show()

# Check imbalance by 'CLASS'
class_counts = label_df['CLASS'].value_counts()
print("Class Counts:")
print(class_counts)

plt.figure(figsize=(8, 6))
sns.countplot(data=label_df, x='CLASS', palette='Set2')
plt.title('Class Distribution')
plt.xlabel('Class')
plt.ylabel('Count')
plt.show()

Resnet 152

In [ ]:
!pip install livelossplot
from livelossplot import PlotLossesKeras
from tensorflow.keras.applications import ResNet50  # Import ResNet152 model

# Initialize ResNet152 model
base_Neural_Net = ResNet50(input_shape=(128, 128, 3), weights='imagenet', include_top=False)

# Define the rest of your model
model = Sequential()
model.add(base_Neural_Net)
model.add(Flatten())
model.add(BatchNormalization())
model.add(Dense(256, kernel_initializer='he_uniform'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))

# Freeze layers of the base ResNet152 model
for layer in base_Neural_Net.layers:
    layer.trainable = False

In [ ]:
# Compile the model
from tensorflow.keras.optimizers import Adam

c1 = PlotLossesKeras()
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy', 'AUC'])

# Train the model
model.fit(x_train, y_train, epochs=10, validation_data=(x_val, y_val),callbacks=[c1, c3], batch_size=16)

In [ ]:
y_pred = model.predict(x_test)

In [ ]:
print("Performance Report:")
y_pred_proba = model.predict(x_test)
y_pred9 = np.argmax(y_pred_proba, axis=1)
y_test9 = np.argmax(y_test, axis=1)  # Assuming y_test is one-hot encoded
target=["B", "M"]

from sklearn import metrics
from sklearn.metrics import confusion_matrix
print('Accuracy score is:', np.round(metrics.accuracy_score(y_test9, y_pred9), 4))
print('Precision score is:', np.round(metrics.precision_score(y_test9, y_pred9, average='weighted'), 4))
print('Recall score is:', np.round(metrics.recall_score(y_test9, y_pred9, average='weighted'), 4))
print('F1 Score is:', np.round(metrics.f1_score(y_test9, y_pred9, average='weighted'), 4))
# ROC AUC score for multi-class classification needs different handling, check the documentation for the correct usage
print('ROC AUC Score is:', np.round(metrics.roc_auc_score(y_test, y_pred_proba, multi_class='ovr', average='weighted'), 4))
print('\t\tClassification Report:\n', metrics.classification_report(y_test9, y_pred9, target_names=target))
# Compute confusion matrix
conf_matrix = confusion_matrix(y_test9, y_pred9)

# Calculate Sensitivity and Specificity for each class
sensitivity = conf_matrix.diagonal() / conf_matrix.sum(axis=1)
specificity = (conf_matrix.sum(axis=1) - conf_matrix.diagonal()) / (conf_matrix.sum(axis=1) + conf_matrix.sum(axis=0) - conf_matrix.diagonal())

# Print Sensitivity and Specificity
for i, label in enumerate(target):
    print(f'Class {label} - Sensitivity: {np.round(sensitivity[i], 4)}, Specificity: {np.round(specificity[i], 4)}')

